In [2]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import google.generativeai as genai

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\USER\AppData\Local\Temp\ipykernel_15172\2608630783.py:5: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as pos

In [3]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
gemini = genai.GenerativeModel("gemini-3-flash-preview")

In [4]:
df = pd.read_csv("../data/credit_risk_dataset.csv")

X = df.drop(columns=["default_risk", "customer_financial_statement"])
y = df["default_risk"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [6]:
pred = model.predict(X_test)
prob = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, pred)
f1 = f1_score(y_test, pred)
auc = roc_auc_score(y_test, prob)

print("Accuracy:", acc)
print("F1 Score:", f1)
print("ROC-AUC:", auc)
print("\nClassification Report:\n", classification_report(y_test, pred))

Accuracy: 0.9333333333333333
F1 Score: 0.8367346938775511
ROC-AUC: 0.9783926799691325

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       193
           1       0.80      0.87      0.84        47

    accuracy                           0.93       240
   macro avg       0.89      0.91      0.90       240
weighted avg       0.94      0.93      0.93       240



In [7]:
importance = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)
top_features = importance.head(10)

print("\nTop Logistic Regression Features:\n")
print(top_features)


Top Logistic Regression Features:

num_late_payments                2.044230
credit_utilization_ratio         0.236073
employment_type_Self-Employed    0.218497
education_level_Master           0.094229
monthly_income                  -0.000008
loan_amount                     -0.000023
loan_duration_months            -0.001739
existing_loans_count            -0.021936
account_tenure_years            -0.023268
age                             -0.027937
dtype: float64


In [8]:
prompt = f"""
You are a financial data analyst.

A Logistic Regression model was trained to predict customer default risk.

Model Evaluation:
Accuracy: {acc:.3f}
F1 Score: {f1:.3f}
ROC-AUC: {auc:.3f}

Top Features:
{top_features.to_string()}

Please provide:

1. A summary of the model performance
2. Insights about which customers are most likely to default
3. An interpretation of the top features in financial terms
4. Recommended actions for a bank or lender
"""

response = gemini.generate_content(prompt).text

print("\nGemini LLM Interpretation:\n")
print(response)


Gemini LLM Interpretation:

As a financial data analyst, I have reviewed the Logistic Regression model results. This model provides a robust framework for credit risk assessment, showing high discriminative power and clear behavioral indicators.

Here is the breakdown of the model performance and the strategic implications.

---

### 1. Summary of Model Performance
The model exhibits **excellent predictive power**, particularly in its ability to distinguish between "good" and "bad" borrowers.

*   **ROC-AUC (0.978):** This is an exceptional score. It indicates that there is a 97.8% probability that the model will rank a randomly chosen defaulting customer higher than a randomly chosen non-defaulting one. It suggests the model is highly effective at ranking risk.
*   **Accuracy (0.933):** The model correctly predicts the outcome for 93.3% of the cases. However, in credit scoring, accuracy can be "boosted" by the fact that most customers don't default.
*   **F1 Score (0.837):** This is 